# Imports

In [1]:
import os
import re
import glob
import json
import numpy as numpy
import pandas as pd
from pprint import pprint 

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#Spacy for lemmatization
import spacy

#Plotting tools
import pyLDAvis
import pyLDAvis.gensim #Important
import matplotlib.pyplot as plt 
#%matplotlib inline

#Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\python\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


# Stopwords

In [2]:
#NLTK stop words
from nltk.corpus import stopwords
stop_words= stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

#### Import Dataset

In [3]:
#basePath = os.path.dirname(os.path.abspath(__file__))
#print(basePath)
df = pd.read_json('59.json')
print(df.text.unique())
df.head()

['女の子戦争が始まらない……。' '@NMqZr18SRFKBX40 座るとコルセットが肋骨に刺さります……(；´Д`）'
 "RT @moomoonbyul: Omg I didn't realised their rainbow blanket!!!!! They bring it to hk! \n#마마무 #솔라 #문별 #용콩별콩 https://t.co/8OlKoFGnvY"
 ...
 'RT @SonandoRoock: Que mimo al alma escuchar lo que significamos para alguien.'
 'だっかっらっ砂糖水飲めないんだよおおおおおお😭😭😭まじ、なにはすの何味とかミルクティー味の水とかこわすぎ普通にクリスタルガイザーかフィジーが絶対正義\nhttps://t.co/VShn9kIRsr'
 '♩#NowPlaying 99.5FM KHEA Radio by https://t.co/SKiha1owgm https://t.co/SKiha1owgm https://t.co/SHmD5H8Ztv']


,contributors,coordinates,created_at,delete,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,...,quoted_status_id_str,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,None,2017-11-09 10:59:00,NaN,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",女の子戦争が始まらない……。,2017-11-09 10:59:00.660,0.0,"{'id': 885564598073442308, 'id_str': '88556459..."
1,NaN,None,2017-11-09 10:59:00,NaN,"[17, 42]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@NMqZr18SRFKBX40 座るとコルセットが肋骨に刺さります……(；´Д`）,2017-11-09 10:59:00.658,0.0,"{'id': 4836244501, 'id_str': '4836244501', 'na..."
2,NaN,None,2017-11-09 10:59:00,NaN,NaN,"{'hashtags': [{'text': '마마무', 'indices': [88, ...","{'media': [{'id': 927099894350757895, 'id_str'...",NaN,0.0,0.0,...,NaN,0.0,0.0,0.0,{'created_at': 'Sun Nov 05 09:07:19 +0000 2017...,"<a href=""http://twitter.com/download/android"" ...",RT @moomoonbyul: Omg I didn't realised their r...,2017-11-09 10:59:00.657,0.0,"{'id': 766092388494368768, 'id_str': '76609238..."
3,NaN,None,2017-11-09 10:59:00,NaN,"[9, 35]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@ko180ex おめでとうございます🙌🎉😊\n自分は県内枠で行きます🙌,2017-11-09 10:59:00.664,0.0,"{'id': 743778210437955584, 'id_str': '74377821..."
4,NaN,None,2017-11-09 10:59:00,NaN,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",もう一回潜りに行こうかな？どのみちサブカ対面だし,2017-11-09 10:59:00.662,0.0,"{'id': 873200024913444864, 'id_str': '87320002..."


#### Clean emails

In [4]:
#Convert to List
data = df.text.values.tolist()

#Remove Emails
#data = [re.sub('\S*@\S*\s?', '',sent) for sent in data]
#Remove new line characters
#data = [re.sub('\s+',' ',sent) for sent in data]
#Remove distracting single qquotes
#data = [re.sub("\'", "", sent) for sent in data]
#Print data from email after cleaning up with REGEX
pprint(data[:1])

['女の子戦争が始まらない……。']


#### Break Sentences into Words

In [5]:
#Break down each sentence into a list of words through TOKENIZATION
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) #Removes Punctuations
data_words = list(sent_to_words(data))
#result from breaking down each sentence into a list of words through TOKENIZATION
print(data_words[:1])

[['女の子戦争か始まらない']]


### Bigram & Trigram Model Building

In [6]:
#Build bigram & Trigram models
bigram = gensim.models.Phrases(data_words,min_count=5,threshold=100) #Higher threshold = fewer phrases
trigram = gensim.models.Phrases(bigram[data_words],threshold=100)
#Faster way to get sentence clubbed as birgram/trigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
#See trigram example
print("-----------------------------")
print(trigram_mod[bigram_mod[data_words[0]]])

c:\python\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


-----------------------------
['女の子戦争か始まらない']


#### Functions: Stopwords, Bi/Trigrams, Lemmatization

In [7]:
#define functions for stopwords, bi/trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out= []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#### Call Functions and display

In [8]:
#remove Stop words
data_words_nostops = remove_stopwords(data_words)
#form bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
#finitialize spacy 'en' model
nlp = spacy.load('en', disable=['parser','ner'])
#Do lemmatization keeping only nouns, adjectives, verbs and adverbs
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#Print data after removing stop words, form bigram and lemmatizing data
print("-------------------------------------")
print(data_lemmatized[:1])

c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


-------------------------------------
[['女の子戦争か始まらない']]


### Create Dictionary and Corpus

In [9]:
#Create Dict
id2word = corpora.Dictionary(data_lemmatized)
#Create Corpus
texts = data_lemmatized
#Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
#View
print(corpus[:1])

#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1)]]


## Build LDA Model

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,
num_topics=20, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
print("TEST: DONE LDA MODEL BUILDING")

# Print the Keyword in the 20 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

TEST: DONE LDA MODEL BUILDING


### Perplexity and coherence

In [11]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
#coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)
print("OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO")


Perplexity:  -10.793718637819676
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO


# Visualize

In [12]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

c:\python\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.357768 -0.002537       1        1  14.891007
10     0.030381  0.062386       2        1   7.188020
2     -0.212405  0.005404       3        1   5.970388
8      0.032295  0.009006       4        1   5.125549
18     0.031591  0.015383       5        1   4.993029
1      0.032355  0.010300       6        1   4.795782
14     0.030163 -0.262385       7        1   4.661534
0      0.032276  0.014169       8        1   4.632896
11     0.031759  0.009719       9        1   4.436546
6      0.031162  0.016018      10        1   4.402662
19     0.031413  0.023289      11        1   4.268495
15     0.031523  0.018403      12        1   4.237484
3      0.032455  0.008925      13        1   4.077193
5      0.032029  0.010143      14        1   3.973758
7      0.030778  0.014400      15        1   3.945404
13     0.032115  0.010144      16        1   3.807915
16     0.032023  0.010140      17        1   3.790500
12     0.031383  0.011205      18        1   3.767180
17     0.032300  0.008973      19        1   3.636161
9      0.032170  0.006916      20        1   3.398489, topic_info=      Category        Freq             Term       Total  loglift  logprob
term                                                                     
5      Default  811.000000            https  811.000000  30.0000  30.0000
4      Default  794.000000               co  794.000000  29.0000  29.0000
496    Default   85.000000                _   85.000000  28.0000  28.0000
1658   Default   26.000000              que   26.000000  27.0000  27.0000
868    Default   29.000000             الله   29.000000  26.0000  26.0000
170    Default   41.000000               la   41.000000  25.0000  25.0000
933    Default   26.000000             para   26.000000  24.0000  24.0000
334    Default   21.000000              get   21.000000  23.0000  23.0000
28     Default   18.000000              amp   18.000000  22.0000  22.0000
232    Default   14.000000               go   14.000000  21.0000  21.0000
848    Default   14.000000              nos   14.000000  20.0000  20.0000
371    Default   10.000000              على   10.000000  19.0000  19.0000
1184   Default   11.000000             take   11.000000  18.0000  18.0000
1188   Default   11.000000              day   11.000000  17.0000  17.0000
4454   Default   13.000000               du   13.000000  16.0000  16.0000
2404   Default   11.000000              الى   11.000000  15.0000  15.0000
355    Default    9.000000              nao    9.000000  14.0000  14.0000
662    Default   12.000000              por   12.000000  13.0000  13.0000
1096   Default   12.000000               rt   12.000000  12.0000  12.0000
2136   Default   10.000000              الا   10.000000  11.0000  11.0000
373    Default    9.000000             good    9.000000  10.0000  10.0000
777    Default   12.000000          youtube   12.000000   9.0000   9.0000
186    Default   10.000000              dia   10.000000   8.0000   8.0000
4785   Default   10.000000             call   10.000000   7.0000   7.0000
3328   Default    9.000000             make    9.000000   6.0000   6.0000
2051   Default   10.000000             much   10.000000   5.0000   5.0000
96     Default    8.000000             need    8.000000   4.0000   4.0000
182    Default    9.000000              new    9.000000   3.0000   3.0000
1100   Default   11.000000              bts   11.000000   2.0000   2.0000
1501   Default    9.000000              ولا    9.000000   1.0000   1.0000
...        ...         ...              ...         ...      ...      ...
355    Topic20    9.007807              nao    9.570427   3.3213  -4.1925
1699   Topic20    5.538898              fan    6.101518   3.2851  -4.6788
157    Topic20    5.188601            think    5.751222   3.2789  -4.7441
46     Topic20    4.795682              com    5.358303   3.2709  -4.8229
10805  Topic20    4.127436     

# mallet

In [13]:
#malletPath
mallet_path ='/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
#Show topics
pprint(ldamallet.show_topics(formatted=False))
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

CalledProcessError: Command '/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\mathe\AppData\Local\Temp\57951d_corpus.txt --output C:\Users\mathe\AppData\Local\Temp\57951d_corpus.mallet' returned non-zero exit status 1.